In [ ]:
#!pip uninstall tensorflow

In [ ]:
#!pip install tensorflow-gpu==2.0.0

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras import layers
print(tf.__version__)
import time
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as mpatches
from sklearn.metrics import roc_auc_score
import random
import itertools
from matplotlib.patches import Ellipse
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
from sklearn.metrics import confusion_matrix


2.3.0


In [2]:
size = 25

In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
tf.compat.v1.Session(config=config)

![title](擷取.PNG)

In [4]:
def gernate_data(n, nu, seed, range_, multi, size):
    ## normal
    np.random.seed(seed=seed)
    y_train = np.ones((int(n*(1-nu)),))
    mean = np.random.uniform(-5, 5, size)
    cov_  = np.random.uniform(0, 1, (size, size))* 3
    cov = np.dot(cov_.T,cov_)
    x_train = np.random.multivariate_normal(mean, cov, (int(n*(1-nu))))
    x_train = x_train.astype('float32')

    
    
    np.random.seed(seed=seed)
    x_trains_ = None ## normal + outlier
    for i in range(multi):
        mean = np.random.uniform(-range_, range_, size)
        cov_ = np.random.randn(size, size)
        cov = np.dot(cov_.T,cov_)
        x_train_ = np.random.multivariate_normal(mean, cov, (int(n*nu/multi)), check_valid  = 'raise')
        try:
            x_trains_ = np.vstack((x_trains_, x_train_))
        except:
            x_trains_ = x_train_
    ## 畫圖
    '''fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(x_train[:, 0], x_train[:, 1],x_train[:, 2], s=0.9, label = 'normal')
    ax.scatter(x_trains_[:,0], x_trains_[:,1], x_trains_[:,2], s=0.9, label = 'outlier')
    plt.legend()
    plt.title('normal + outlier')
    plt.show()'''

    y_trains_ = np.zeros(((len(x_trains_),)))
    x_train_mix = np.vstack((x_trains_, x_train)).astype('float32')
    y_train_mix = np.hstack((y_trains_, y_train)).astype('float32')
    
    return x_train_mix, y_train_mix

## 參數設定

In [5]:

z_dim = 5
h_dim = [10, 5]
regular = 1e-5
max_lr = 1e-6
base_lr = 1e-5


n_epochs = 1000

optimizer = tf.keras.optimizers.Adam(lr = base_lr)

In [6]:
def make_ae_model():

    inputs = tf.keras.Input(shape=(size,), name='inputs')

    x = tf.keras.layers.Dense(h_dim[0], activation='relu', use_bias=True)(inputs)
    x = tf.keras.layers.Dense(h_dim[1], activation='linear', use_bias=False, name = 'hidden')(x)
    x = tf.keras.layers.Dense(h_dim[0], activation='relu', use_bias=True)(x)
    x = tf.keras.layers.Dense(size, activation='linear', use_bias=True)(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)

    return model

ae_model = make_ae_model()
ae_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                260       
_________________________________________________________________
hidden (Dense)               (None, 5)                 50        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                60        
_________________________________________________________________
dense_2 (Dense)              (None, 25)                275       
Total params: 645
Trainable params: 645
Non-trainable params: 0
_________________________________________________________________


In [7]:
def make_model(model):
    
    model.trainable = True
    x = model.get_layer('hidden').output
    
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model_out = tf.keras.Model(inputs = model.input, outputs = outputs)
    
    return model_out

model = make_model(ae_model)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                260       
_________________________________________________________________
hidden (Dense)               (None, 5)                 50        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 316
Trainable params: 316
Non-trainable params: 0
_________________________________________________________________


## Train SA-SVDD

In [11]:
class train_classfication():
    
    def __init__(self , binary_model, train_dataset, nu ):
        
        self.nu = nu
        self.binary_model = binary_model
        self.optimizer = tf.keras.optimizers.Adam(lr = 1e-2, epsilon = 1e-6)
        self.train_dataset = train_dataset

    def loss(self, batch_x, batch_y):
        
        avg_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=batch_y, logits=batch_x)
        end_loss = tf.reduce_mean(avg_loss)

        return end_loss

    @tf.function
    def train_model(self, batch_x, batch_y):
        with tf.GradientTape() as tape:
            output = self.binary_model(batch_x, training=True)
            output = tf.squeeze(output)
            output_loss = self.loss(output, batch_y)

        grads = tape.gradient(output_loss, self.binary_model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.binary_model.trainable_variables))

        return output_loss
    
    def train( self, lr_milestone = 15):
        
        
        pre_loss = np.inf
        loss = 0.
        #while abs(loss - pre_loss) > 1e-6:
        for i in range(100):
            epoch_ae_loss_avg = tf.metrics.Mean()

            for batch, (batch_x, batch_y) in enumerate(train_dataset):

                ae_loss = self.train_model(batch_x, batch_y)
                epoch_ae_loss_avg(ae_loss)
            #loss_list.append(epoch_ae_loss_avg.result())
        
        self.binary_model.save_weights('model_{}'.format(
            self.nu))
        
    def score(self, data):
        self.binary_model.load_weights('model_{}' .format(
            self.nu))
        output = self.binary_model(data, training = False)
        return output
    
    def train_result(self, x_novali, y_novali, show_img = False):
        
        test_auc = 0.
        confusionmatrix = 0.
        if show_img:
            scores = self.score(x_novali).numpy()
            test_auc = self.result(scores, y_novali)

            
            
        return radius_end, test_auc, confusionmatrix
    
    def result(self, scores, y_test):
        
        y_true = y_test
        test_auc = roc_auc_score(y_true, scores)

        return test_auc 
    
    def test_result(self, x_test, y_test):
        
        scores = self.score(x_test).numpy()
        test_auc = self.result(scores, y_test)

        return test_auc

In [12]:
nu = 0.1
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_mix, y_train_mix))
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [13]:
result = np.zeros((25, 3))
warm_up_n_epochs = 10
for i in range(25):
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    ae_model.fit(x_train_mix, x_train_mix, epochs = 30, verbose = 0)
    ## 訓練 SVDD
    binary_model = make_model(ae_model)
    binary = train_classfication(binary_model, train_dataset, nu)
    binary.train(x_train_mix)
    auc  = binary.test_result(x_test_mix, y_test_mix)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.9187667227833894
epoch =  1 auc =  0.9203156004489338
epoch =  2 auc =  0.9475486531986532
epoch =  3 auc =  0.9190302469135804
epoch =  4 auc =  0.9082379349046016
epoch =  5 auc =  0.8640042087542088
epoch =  6 auc =  0.920734960718294
epoch =  7 auc =  0.9034895622895623
epoch =  8 auc =  0.905008810325477
epoch =  9 auc =  0.8968582491582493
epoch =  10 auc =  0.9205410213243547
epoch =  11 auc =  0.9195152076318742
epoch =  12 auc =  0.8865420314253648
epoch =  13 auc =  0.9325700336700335
epoch =  14 auc =  0.8947742985409651
epoch =  15 auc =  0.9098488776655443
epoch =  16 auc =  0.9219805274971943
epoch =  17 auc =  0.8865034231200897
epoch =  18 auc =  0.894890684624018
epoch =  19 auc =  0.9280808641975309
epoch =  20 auc =  0.8988755331088665
epoch =  21 auc =  0.9053321548821549
epoch =  22 auc =  0.9064182379349046
epoch =  23 auc =  0.9156694163860832
epoch =  24 auc =  0.9177734006734007


In [14]:
np.mean(result, 0)

array([12.        ,  0.90973243, 10.30937741])

In [15]:
np.std(result, 0)

array([7.21110255, 0.01690188, 0.21061531])

In [16]:
nu = 0.2
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_mix, y_train_mix))
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [17]:
result = np.zeros((25, 3))
warm_up_n_epochs = 10
for i in range(25):
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    ae_model.fit(x_train_mix, x_train_mix, epochs = 30, verbose = 0)
    ## 訓練 SVDD
    binary_model = make_model(ae_model)
    binary = train_classfication(binary_model, train_dataset, nu)
    binary.train(x_train_mix)
    auc  = binary.test_result(x_test_mix, y_test_mix)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.9376436403508772
epoch =  1 auc =  0.9298505012531328
epoch =  2 auc =  0.9118032268170426
epoch =  3 auc =  0.9410347744360902
epoch =  4 auc =  0.9287085839598997
epoch =  5 auc =  0.9210345864661654
epoch =  6 auc =  0.9194416040100252
epoch =  7 auc =  0.9262184523809525
epoch =  8 auc =  0.8862589598997493
epoch =  9 auc =  0.9156498746867168
epoch =  10 auc =  0.9193897556390977
epoch =  11 auc =  0.9426782894736842
epoch =  12 auc =  0.8988037593984962
epoch =  13 auc =  0.8898232142857143
epoch =  14 auc =  0.9508198621553885
epoch =  15 auc =  0.9285719611528821
epoch =  16 auc =  0.917617731829574
epoch =  17 auc =  0.894420112781955
epoch =  18 auc =  0.9341357142857142
epoch =  19 auc =  0.9184058897243108
epoch =  20 auc =  0.9177897869674186
epoch =  21 auc =  0.9044556704260651
epoch =  22 auc =  0.8727708020050127
epoch =  23 auc =  0.9119331453634085
epoch =  24 auc =  0.9175361528822056


In [18]:
np.mean(result, 0)

array([12.        ,  0.91747184, 10.21513924])

In [19]:
np.std(result, 0)

array([7.21110255, 0.01826474, 0.14463275])

In [20]:
nu = 0.3
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_mix, y_train_mix))
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [21]:
result = np.zeros((25, 3))
warm_up_n_epochs = 10
for i in range(25):
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    ae_model.fit(x_train_mix, x_train_mix, epochs = 30, verbose = 0)
    ## 訓練 SVDD
    binary_model = make_model(ae_model)
    binary = train_classfication(binary_model, train_dataset, nu)
    binary.train(x_train_mix)
    auc  = binary.test_result(x_test_mix, y_test_mix)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.9040804047619048
epoch =  1 auc =  0.9528693333333332
epoch =  2 auc =  0.9361608809523809
epoch =  3 auc =  0.07717626190476191
epoch =  4 auc =  0.9447001190476192
epoch =  5 auc =  0.9311743571428572
epoch =  6 auc =  0.9343561904761905
epoch =  7 auc =  0.8832826190476191
epoch =  8 auc =  0.916524761904762
epoch =  9 auc =  0.9210520238095239
epoch =  10 auc =  0.9115696428571429
epoch =  11 auc =  0.9318514523809525
epoch =  12 auc =  0.9263794285714286
epoch =  13 auc =  0.9035899285714286
epoch =  14 auc =  0.9431405714285716
epoch =  15 auc =  0.9624005714285714
epoch =  16 auc =  0.926412
epoch =  17 auc =  0.9403735476190477
epoch =  18 auc =  0.9084170952380952
epoch =  19 auc =  0.9186935476190476
epoch =  20 auc =  0.9222127142857143
epoch =  21 auc =  0.9270886904761905
epoch =  22 auc =  0.8854538333333333
epoch =  23 auc =  0.9146611428571428
epoch =  24 auc =  0.9441179285714285


In [22]:
np.mean(result, 0)

array([12.        ,  0.89070956, 10.34319324])

In [23]:
np.std(result, 0)

array([7.21110255, 0.16710555, 0.15789616])